<a href="https://colab.research.google.com/github/apchavezr/-Analisis_Grandes_Volumenes_Datos/blob/main/Kafka_PySpark_Hive_Airflow_(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Flujo distribuido: Kafka + PySpark + Hive + Airflow

Este notebook presenta un ejemplo simplificado del flujo de procesamiento distribuido utilizando PySpark para consumir datos desde Apache Kafka, transformarlos y almacenarlos en formato Parquet, simulando una integración con Apache Hive y Apache Airflow.

**Componentes cubiertos:**
- Kafka: simulado como fuente de datos.
- PySpark: procesamiento de datos con Structured Streaming.
- Hive: simulado mediante almacenamiento en formato Parquet.
- Airflow: se describe cómo se orquestaría con DAG (fuera del notebook).


In [1]:
# Configuración inicial de PySpark
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col
from pyspark.sql.types import StructType, StringType, TimestampType

spark = SparkSession.builder \
    .appName("Kafka_PySpark_Integration") \
    .config("spark.sql.shuffle.partitions", "2") \
    .getOrCreate()

## Esquema del mensaje desde Kafka (simulado)

In [2]:
# Simulación del esquema de datos
schema = StructType() \
    .add("sensor_id", StringType()) \
    .add("zona", StringType()) \
    .add("valor", StringType()) \
    .add("timestamp", TimestampType())

## Lectura desde Kafka (ejemplo real requeriría acceso a Kafka)

In [3]:
# kafkaDF = spark.readStream \
#     .format("kafka") \
#     .option("kafka.bootstrap.servers", "localhost:9092") \
#     .option("subscribe", "eventos") \
#     .load()

# jsonDF = kafkaDF.selectExpr("CAST(value AS STRING)")
# dataDF = jsonDF.select(from_json(col("value"), schema).alias("data")).select("data.*")

## Simulación del DataFrame transformado

In [4]:
# Simulación local de datos
from datetime import datetime
data = [("sensor-01", "Usaquén", "35", datetime.now()),
        ("sensor-02", "Chapinero", "48", datetime.now())]
df = spark.createDataFrame(data, schema=schema)

# Transformación: convertir valor a entero y agregar columna
df_transformed = df.withColumn("valor", col("valor").cast("int"))
df_transformed.show()

+---------+---------+-----+--------------------+
|sensor_id|     zona|valor|           timestamp|
+---------+---------+-----+--------------------+
|sensor-01|  Usaquén|   35|2025-04-21 18:09:...|
|sensor-02|Chapinero|   48|2025-04-21 18:09:...|
+---------+---------+-----+--------------------+



## Escritura en formato Parquet (para consumo posterior en Hive)

In [5]:
df_transformed.write.mode("overwrite").parquet("datos_sensores")

## Explicación paso a paso del flujo

**1. Configuración inicial de PySpark:**
Se crea una sesión de Spark con parámetros básicos. Esta sesión permite ejecutar operaciones distribuidas sobre conjuntos de datos de gran tamaño.

**2. Definición del esquema:**
Se especifica el formato de los datos que llegarán desde Kafka. Esto es clave para interpretar correctamente el contenido de los mensajes y realizar transformaciones posteriores.

**3. Lectura desde Kafka:**
Aunque se comenta en este ejemplo (por no tener acceso a un clúster Kafka en ejecución), se ilustra cómo se usaría `readStream` para consumir datos de un topic de Kafka. La transformación con `from_json` permite convertir los datos en estructuras tabulares.

**4. Simulación del DataFrame:**
Dado que no estamos en un entorno con Kafka en tiempo real, se crea manualmente un conjunto de datos simulados para representar los datos que llegarían desde Kafka. Se transforma la columna `valor` a entero para su análisis.

**5. Escritura en Parquet:**
Los datos transformados se almacenan en formato Parquet, optimizado para análisis posteriores en sistemas como Apache Hive. Hive puede leer directamente estos archivos y tratarlos como tablas para consultas SQL.


## Conclusión

Este flujo simulado permite comprender cómo una arquitectura moderna basada en tecnologías distribuidas puede gestionar datos en tiempo real desde su ingestión hasta su análisis.

- **Kafka** actúa como buffer resiliente de datos en tiempo real.
- **PySpark** procesa los datos con alta eficiencia mediante transformación distribuida.
- **Hive** permite analizar los datos de forma declarativa y estructurada.
- **Airflow** (aunque no se ejecuta directamente en este notebook) puede orquestar todo el pipeline y asegurar que cada componente se ejecute según lo planeado.

Este flujo es aplicable en escenarios reales como monitoreo ambiental, análisis de tráfico o procesamiento de logs, y puede escalar horizontalmente en entornos como AWS EMR, Databricks o GCP Dataproc.